In [1]:
from lxml import etree
import pandas as pd
from io import StringIO

In [2]:
doc = etree.parse("fpds_feed_sample_1_org.xml")

In [3]:
xslt_tree = etree.parse("fpds_feed_v3.xsl")
transform = etree.XSLT(xslt_tree)

In [4]:
result = transform(doc)

In [5]:
csv_str =str(result)
csv_str

'feePaidForUseOfService\t0.00\t0.00\t0.00\t0.00\t0.00\t0.00\t0.00\t0.00\t0.00\t0.00\ntotalEstimatedOrderValue\t\t\t\t\t\t\t\t\t\t\ncontractActionType\tC\tC\tC\tC\tC\tC\tC\tC\tA\tC\ncontractActionType-description\tDELIVERY ORDER\tDELIVERY ORDER\tDELIVERY ORDER\tDELIVERY ORDER\tDELIVERY ORDER\tDELIVERY ORDER\tDELIVERY ORDER\tDELIVERY ORDER\tBPA CALL\tDELIVERY ORDER\ncontractActionType-part8OrPart13\t\t\t\t\t\t\t\t\tPART13\t\ntypeOfContractPricing\tU\tU\tU\tU\tU\tU\tJ\tY\tJ\tJ\ntypeOfContractPricing-description\tCOST PLUS FIXED FEE\tCOST PLUS FIXED FEE\tCOST PLUS FIXED FEE\tCOST PLUS FIXED FEE\tCOST PLUS FIXED FEE\tCOST PLUS FIXED FEE\tFIRM FIXED PRICE\tTIME AND MATERIALS\tFIRM FIXED PRICE\tFIRM FIXED PRICE\nreasonForModification\t\t\t\t\t\t\t\t\t\tC\nreasonForModification-description\t\t\t\t\t\t\t\t\t\tFUNDING ONLY ACTION\nmajorProgramCode\t\t\t\t\t\t\t\t\t\tINFORMATION TECHNOLOGY SERVICES\n'

In [6]:
mystr = StringIO(csv_str)

df = pd.read_csv(mystr, sep="\t", header=None, index_col=0)
df.tail(10)

,1,2,3,4,5,6,7,8,9,10
0,,,,,,,,,,
feePaidForUseOfService,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
totalEstimatedOrderValue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
contractActionType,C,C,C,C,C,C,C,C,A,C
contractActionType-description,DELIVERY ORDER,DELIVERY ORDER,DELIVERY ORDER,DELIVERY ORDER,DELIVERY ORDER,DELIVERY ORDER,DELIVERY ORDER,DELIVERY ORDER,BPA CALL,DELIVERY ORDER
contractActionType-part8OrPart13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PART13,NaN
typeOfContractPricing,U,U,U,U,U,U,J,Y,J,J
typeOfContractPricing-description,COST PLUS FIXED FEE,COST PLUS FIXED FEE,COST PLUS FIXED FEE,COST PLUS FIXED FEE,COST PLUS FIXED FEE,COST PLUS FIXED FEE,FIRM FIXED PRICE,TIME AND MATERIALS,FIRM FIXED PRICE,FIRM FIXED PRICE
reasonForModification,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C
reasonForModification-description,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FUNDING ONLY ACTION


In [7]:
df_new = df.transpose(copy=True)
df_new.iloc[:, -10:]

,feePaidForUseOfService,totalEstimatedOrderValue,contractActionType,contractActionType-description,contractActionType-part8OrPart13,typeOfContractPricing,typeOfContractPricing-description,reasonForModification,reasonForModification-description,majorProgramCode
1,0.00,NaN,C,DELIVERY ORDER,NaN,U,COST PLUS FIXED FEE,NaN,NaN,NaN
2,0.00,NaN,C,DELIVERY ORDER,NaN,U,COST PLUS FIXED FEE,NaN,NaN,NaN
3,0.00,NaN,C,DELIVERY ORDER,NaN,U,COST PLUS FIXED FEE,NaN,NaN,NaN
4,0.00,NaN,C,DELIVERY ORDER,NaN,U,COST PLUS FIXED FEE,NaN,NaN,NaN
5,0.00,NaN,C,DELIVERY ORDER,NaN,U,COST PLUS FIXED FEE,NaN,NaN,NaN
6,0.00,NaN,C,DELIVERY ORDER,NaN,U,COST PLUS FIXED FEE,NaN,NaN,NaN
7,0.00,NaN,C,DELIVERY ORDER,NaN,J,FIRM FIXED PRICE,NaN,NaN,NaN
8,0.00,NaN,C,DELIVERY ORDER,NaN,Y,TIME AND MATERIALS,NaN,NaN,NaN
9,0.00,NaN,A,BPA CALL,PART13,J,FIRM FIXED PRICE,NaN,NaN,NaN
10,0.00,NaN,C,DELIVERY ORDER,NaN,J,FIRM FIXED PRICE,C,FUNDING ONLY ACTION,INFORMATION TECHNOLOGY SERVICES


In [ ]:
df_new.shape

In [ ]:
# output dataframe to a .csv file
df_new.to_csv("fpds_feed_result.csv", index=False)